## **Step 1: Setting Up the Model for Face Sentiment Analysis**

### 1.1. Instal Required Libraries
```python 
- torch==1.9.1
- torchvision==0.10.1
- numpy==1.21.0
- pandas==1.3.0
- Pillow==8.2.0
- scikit-learn==0.24.2

In [7]:
!pip install flask

  Obtaining dependency information for flask from https://files.pythonhosted.org/packages/36/42/015c23096649b908c809c69388a805a571a3bea44362fe87e33fc3afa01f/flask-3.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for blinker>=1.6.2 from https://files.pythonhosted.org/packages/fa/2a/7f3714cbc6356a0efec525ce7a0613d581072ed6eb53eb7b9754f33db807/blinker-1.7.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/99.7 kB ? eta -:--:--
   ---------------------------------------- 99.7/99.7 kB 2.8 MB/s eta 0:00:00


### 1.2. Import Torch Required Libraries

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

### 1.3. Checking for GPU Availability

Next, we'll check if a GPU is available for accelerated training. If a GPU is detected, we'll set the device to CUDA; otherwise, we'll use the CPU for computations. This step ensures that we can leverage hardware acceleration if it's accessible. Let's proceed with the implementation.


In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU ({torch.cuda.get_device_name(0)}) is available")
else:
    device = torch.device("cpu")
    print("GPU not available, using CPU")

GPU not available, using CPU


## **Step 2: Data Preparation**

In this step, we'll focus on preparing the dataset for training. This includes loading the necessary libraries and performing tasks such as reading image data and organizing it for later use in the model. Let's move on to the code implementation.

In [3]:
import os
import pandas as pd
import numpy as np
from PIL import Image

### 2.1. Setting Directory Locations

Now, let's define the directory locations for the training and testing datasets. These paths will be used to access the image data for model training and evaluation.


In [4]:
# Directory Location
TRAIN_DIR = 'dataset/images/images/train'
TEST_DIR = 'dataset/images/images/test'

### 2.2. Creating Custom Dataset and DataFrames

To effectively handle our image data, we'll create a custom dataset class and corresponding data frames for both training and testing sets.


In [5]:

class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('L')  # Convert to grayscale
        label = int(self.dataframe.iloc[idx, 1])

        if self.transform:
            image = self.transform(image)

        return image, label

def create_dataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return pd.DataFrame({'image': image_paths, 'label': labels})

# Create DataFrames
train = create_dataframe(TRAIN_DIR)
test = create_dataframe(TEST_DIR)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed
angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


### 2.3. Data Transformations

To prepare the data for training, we'll apply a series of transformations to the images. These transformations help in standardizing the format and preparing the data for the model.


In [6]:
# Data transformations
transform = transforms.Compose([
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
])

## **Step 3: Label Encoding**

In this step, we perform label encoding to convert categorical labels into numerical format. This is important for training the model as it requires numerical input.

In [7]:
from sklearn.preprocessing import LabelEncoder

# Create LabelEncoder instance
le = LabelEncoder()

# Fit and transform the labels in your train and test DataFrames
train['label'] = le.fit_transform(train['label'])
test['label'] = le.transform(test['label'])


In [14]:
print(f'Total Train Dataset: {len(train)}')
print(f'Total Test Dataset : {len(test)}')

Total Train Dataset: 20649
Total Test Dataset : 14668


## **Step 4: Load Data into DataLoader**

Now that we have our dataset ready, we create DataLoaders. DataLoaders help in efficient handling of data during training and testing.

In [9]:
# Load data into DataLoader
train_dataset = CustomDataset(dataframe=train, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

test_dataset = CustomDataset(dataframe=test, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

## **Step 5: Define a Simple CNN Architecture**

In this step, you have defined a simple Convolutional Neural Network (CNN) architecture for your Face Sentiment Analysis task. This architecture is designed to process grayscale images and classify them into one of seven emotion categories.

The architecture comprises the following components:

1. **Convolutional Layers:**
   - `conv1`: 16 filters, 3x3 kernel size, ReLU activation, padding = 1.
   - `conv2`: 32 filters, 3x3 kernel size, ReLU activation, padding = 1.
   - `conv3`: 64 filters, 3x3 kernel size, ReLU activation, padding = 1.

2. **Max Pooling Layers:**
   - Max pooling with 2x2 kernel size and stride of 2 is applied after each convolutional layer.

3. **Fully Connected Layers:**
   - `fc1`: 512 output units, ReLU activation.
   - `fc2`: Output layer with units equal to the number of classes (in this case, 7 for the 7 emotions).

4. **Forward Pass:**
   - The input image is passed through the convolutional layers with ReLU activation and max pooling.
   - The output is then flattened and passed through the fully connected layers.

This architecture serves as a solid foundation for your Face Sentiment Analysis model. In the next steps, you will proceed with training, evaluation, and testing.

In [10]:
# Define a simpler CNN architecture
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=7):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 6 * 6, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 64 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## **Step 6: Initialize the Model**

In this step, you have initialized your Simple CNN model. Additionally, you have checked if a CUDA-enabled GPU is available and moved the model to the GPU for faster training.

```python
# Initialize your model
model = SimpleCNN()

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device (GPU or CPU)
model.to(device)
```

This ensures that your model is ready for training on the available hardware resources. Next, you'll move on to the training phase.

In [11]:
# Initialize your model
model = SimpleCNN()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

SimpleCNN(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2304, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=7, bias=True)
)

## **Step 7: Define Loss Function and Optimizer**

In this step, you've defined the loss function and optimizer for your model.

```python
# Define loss function
criterion = nn.CrossEntropyLoss()

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)
```

The chosen loss function (`nn.CrossEntropyLoss`) is commonly used for multi-class classification tasks. The optimizer (`optim.Adam`) is an efficient optimization algorithm that adapts the learning rate during training.

You're now set to begin the training process. Proceed to Step 10 for the training loop.

In [12]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## **Step 8: Model Evaluation and Performance Analysis**

In this step, we've implemented the training loop for your Face Sentiment Analysis model. This loop will iterate through the specified number of epochs and perform the following operations:

In [13]:
# Assuming you have dataloaders defined as train_loader and test_loader
# Initialize lists to store accuracies and losses
train_accuracies = []
train_losses = []
val_accuracies = []
val_losses = []

# Training loop
num_epochs = 30

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    train_accuracy = correct_train / total_train

    # Validation/Test
    model.eval()
    correct_val = 0
    total_val = 0
    val_loss = 0.0

    with torch.no_grad():
        for inputs, labels in test_loader:  # Assuming you have a separate validation loader
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_accuracy = correct_val / total_val
    val_loss /= len(test_loader)

    # Append accuracies and losses to their respective lists
    train_accuracies.append(train_accuracy)
    train_losses.append(running_loss / len(train_loader))
    val_accuracies.append(val_accuracy)
    val_losses.append(val_loss)

    print(f'Epoch [{epoch + 1}/{num_epochs}], '
          f'Training Loss: {running_loss / len(train_loader):.4f}, '
          f'Training Accuracy: {train_accuracy:.2f}, '
          f'Validation Loss: {val_loss:.4f}, '
          f'Validation Accuracy: {val_accuracy:.2f}')


Epoch [1/30], Training Loss: 1.6941, Training Accuracy: 0.33, Validation Loss: 1.6193, Validation Accuracy: 0.39
Epoch [2/30], Training Loss: 1.4944, Training Accuracy: 0.45, Validation Loss: 1.4899, Validation Accuracy: 0.44
Epoch [3/30], Training Loss: 1.3823, Training Accuracy: 0.49, Validation Loss: 1.3815, Validation Accuracy: 0.48
Epoch [4/30], Training Loss: 1.2923, Training Accuracy: 0.52, Validation Loss: 1.3528, Validation Accuracy: 0.49
Epoch [5/30], Training Loss: 1.2185, Training Accuracy: 0.55, Validation Loss: 1.2762, Validation Accuracy: 0.52
Epoch [6/30], Training Loss: 1.1573, Training Accuracy: 0.57, Validation Loss: 1.2640, Validation Accuracy: 0.53
Epoch [7/30], Training Loss: 1.0831, Training Accuracy: 0.60, Validation Loss: 1.1713, Validation Accuracy: 0.57
Epoch [8/30], Training Loss: 1.0127, Training Accuracy: 0.63, Validation Loss: 1.1486, Validation Accuracy: 0.58
Epoch [9/30], Training Loss: 0.9356, Training Accuracy: 0.66, Validation Loss: 1.1451, Validatio

In [18]:
# Checking Accuracy and Running Loss
total_accuracy = 100 * correct_train / total_train

# Print total loss and total accuracy
print(f'Total Loss: {running_loss:.4f}, Total Accuracy: {total_accuracy:.2f}%')


Total Loss: 9.8170, Total Accuracy: 98.32%


## **Final: Save the Model**

In [16]:
# Assuming your model is named 'SimpleCNN'
# Save the model
torch.save(model.state_dict(), 'emotiondetector.h5')
